In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
from PIL import Image
from bs4 import BeautifulSoup
import torch
import torchvision.models as models
import torch.optim as optim
import torchvision

In [ ]:
images_path = "/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Dataset/images"
annots_path = "/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Dataset/annotations"

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
def make_box(obj):
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)

    return [xmin, ymin, xmax, ymax]

def encode_category(obj):
    category = obj.find('name').text
    if category == "without_mask":
        return 0
    elif category == "with_mask":
        return 1
    else: # category == "mask_weared_incorrect"
        return 2

def make_annot(idx, annot_path):
    file = open(annot_path)
    soup = BeautifulSoup(file, 'xml')
    objects = soup.find_all('object')
    num_objects = len(objects)

    boxes = []
    labels = []
    for obj in objects:
        boxes.append(make_box(obj))
        labels.append(encode_category(obj))
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.as_tensor(labels, dtype=torch.int64)

    annot = {}
    annot["boxes"] = boxes
    annot["labels"] = labels

    return annot

In [ ]:
class MaskDataset(torch.utils.data.Dataset):
    def __init__(self, transforms=None):
        self.transforms = transforms
        self.images = glob.glob(f"{images_path}/*.png")

    def __getitem__(self, idx):
        img_path = f"{images_path}/maksssksksss{idx}.png"
        ant_path = f"{annots_path}/maksssksksss{idx}.xml"
        image = Image.open(img_path).convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)
        annot = make_annot(idx, ant_path)

        return image, annot
        

    def __len__(self):
        return len(self.images)

In [ ]:
num_classes = 3

model = models.detection.fasterrcnn_resnet50_fpn(weights='COCO_V1')
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
transforms = torchvision.transforms.Compose([
      torchvision.transforms.ToTensor()
])

def custom_collate_fn(batch):
    return tuple(zip(*batch))

maskdataset = MaskDataset(transforms)
# data_loader = torch.utils.data.DataLoader(maskdataset)
data_loader = torch.utils.data.DataLoader(maskdataset,
                                          batch_size=4,
                                          collate_fn=custom_collate_fn)

In [ ]:
learning_rate = 0.001
num_epochs = 1000

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

filepath = "/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector"
modelpath = "model"

model.load_state_dict(torch.load(f"{filepath}/{modelpath}/model_state_dict.pt"))
optimizer.load_state_dict(torch.load(f"{filepath}/{modelpath}/optim_state_dict.pt"))

model.train()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
import time

start = time.time()

def Total_Loss(loss):
  loss_objectness = loss['loss_objectness']
  loss_rpn_box_reg = loss['loss_rpn_box_reg']
  loss_classifier = loss['loss_classifier']
  loss_box_reg = loss['loss_box_reg']

  rpn_total = loss_objectness + 10*loss_rpn_box_reg
  fast_rcnn_total = loss_classifier + 1*loss_box_reg

  total_loss = rpn_total + fast_rcnn_total

  return total_loss

for epoch in range(num_epochs):
    epoch_loss = 0
    len_data_loader = len(data_loader)
    for i, (images, annotations) in enumerate(data_loader):
        print(f"({i}/{len_data_loader})")
        images = [image.to(device) for image in images]
        annotations = [{key: val.to(device) for key, val in obj.items()} for obj in annotations]

        loss_dict = model(images, annotations)
        # total_loss = Total_Loss(loss)
        total_loss = sum(loss for loss in loss_dict.values()) 

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss
    print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || [{epoch}/{num_epochs}], Loss = {epoch_loss.data:.2f}""")
print("-------End------")
elapsed = time.time() - start
print(f"End of training, elapsed time : {elapsed // 60} min {elapsed % 60} sec.")

(0/214)
(1/214)
(2/214)
(3/214)
(4/214)
(5/214)
(6/214)
(7/214)
(8/214)
(9/214)
(10/214)
(11/214)
(12/214)
(13/214)
(14/214)
(15/214)
(16/214)
(17/214)


KeyboardInterrupt: ignored

In [ ]:
best_model_state = model.state_dict()
best_optim_state = optimizer.state_dict()

In [ ]:
filepath = "/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector"
modelpath = "model"
if best_model_state is not None and best_optim_state is not None:
    torch.save(best_model_state, f"{filepath}/{modelpath}/model_state_dict.pt")
    torch.save(best_optim_state, f"{filepath}/{modelpath}/optim_state_dict.pt")
    # model.load_state_dict(best_model_state)
    # torch.save(model, f"{filepath}/{experiment}/best_model.pt")
    print("Successfully saved.")

Successfully saved.
